if want to try tensor flow:

source: https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/multilayer_perceptron.py

In [2]:
import numpy as np
import numpy.matlib
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Get Data

In [18]:
# signalOutput-coco_3_cv_3_netAng_30_twc_10_tfidfNoPro_pronoun_bin_1-OQD9U4.csv
rawSignals = pd.read_csv('signalOutput-coco_3_cv_3_netAng_30_twc_10_tfidfNoPro_pronoun_bin_1-OQD9U4.csv', index_col=0)

In [19]:
docRanks = pd.read_csv('docRanks.csv')
docRanks[['groupId']] = docRanks[['groupName']] 

In [20]:
data = pd.merge(rawSignals.ix[:,:], docRanks.ix[:,1:] )

In [21]:
data = data.drop('files', 1)
data = data.drop('timeRun', 1)
data = data.drop('keywords', 1)

In [22]:
# change to vector of binary scores
data['one'] = 0
condition = data['rank'] == 1
data.loc[condition, 'one'] = 1
data.loc[~condition, 'one'] = 0

data['two'] = 0
condition = data['rank'] == 2
data.loc[condition, 'two'] = 1
data.loc[~condition, 'two'] = 0

data['three'] = 0
condition = data['rank'] == 3
data.loc[condition, 'three'] = 1
data.loc[~condition, 'three'] = 0

data['four'] = 0
condition = data['rank'] == 4
data.loc[condition, 'four'] = 1
data.loc[~condition, 'four'] = 0

data['five'] = 0
condition = data['rank'] == 5
data.loc[condition, 'five'] = 1
data.loc[~condition, 'five'] = 0

data['six'] = 0
condition = data['rank'] == 6
data.loc[condition, 'six'] = 1
data.loc[~condition, 'six'] = 0

data['seven'] = 0
condition = data['rank'] == 7
data.loc[condition, 'seven'] = 1
data.loc[~condition, 'seven'] = 0

data['eight'] = 0
condition = data['rank'] == 8
data.loc[condition, 'eight'] = 1
data.loc[~condition, 'eight'] = 0

data['nine'] = 0
condition = data['rank'] == 9
data.loc[condition, 'nine'] = 1
data.loc[~condition, 'nine'] = 0

In [34]:
x = data.ix[:,1:18]
y = data.ix[:, 19:28]

In [36]:
x = x.as_matrix()
y = y.as_matrix()

In [37]:
# STANDARDIZE THE X-MATRIX - why do we do this?
x = StandardScaler().fit_transform(x)

Split into training and testing

In [38]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=42)

Define Functions

In [13]:
# loss function
class nn_MSECriterion:
    def forward(self, predictions, labels):
        return np.sum(np.square(predictions - labels))
        
    def backward(self, predictions, labels):
        num_samples = labels.shape[0]
        return num_samples * 2 * (predictions - labels)

# activation function
class nn_ReLU:
    def forward(self, x):
        if x > 0:
            return x
        else:
            return 0
    
    def inverse:# to make predictions
    
    def backward(self, x, gradOutput):
        if x > 0:
            return np.multiply(1, gradOutput)
        else:
            return np.multiply(0, gradOutput)

# linear layer
class nn_Linear:
    def __init__(self, input_dim, output_dim):
        # Initialized with random numbers from a gaussian N(0, 0.001)
        self.weight = np.matlib.randn(input_dim, output_dim) * 0.01
        self.bias = np.matlib.randn((1, output_dim)) * 0.01
        self.gradWeight = np.zeros_like(self.weight)
        self.gradBias = np.zeros_like(self.bias)
        
    def forward(self, x):
        return np.dot(x, self.weight) + self.bias
    
    def backward(self, x, gradOutput):
        # dL/dw = dh/dw * dL/dv
        self.gradWeight = np.dot(x.T, gradOutput)
        # dL/db = dh/db * dL/dv
        self.gradBias = np.copy(gradOutput)
        # return dL/dx = dh/dx * dL/dv
        return np.dot(gradOutput, self.weight.T)
    
    def getParameters(self):
        params = [self.weight, self.bias]
        gradParams = [self.gradWeight, self.gradBias]
        return params, gradParams

Train Network

In [14]:
hiddenlayer = 5
learningRate = 0.1

model = {}
model['layer1'] = nn_Linear(17,hiddenlayer)
model['layer2'] = nn_Linear(hiddenlayer,9)
model['relu'] = nn_ReLu()
model['loss'] = nn_MSECriterion()

preds = []

for epoch in range(0, 1000):
    loss = 0
    for i in range(0, x_train.shape[0]):
        xi = x[i:i+1, :]
        yi = y[i:i+1, :]

        # Forward.
        a0 = model['layer1'].forward(xi)
        a1 = model['relu'].forward(a0)
        a2 = model['layer2'].forward(a1)
        a3 = model['relu'].forward(a2)
        if epoch == 1000: 
            preds = preds + [float(a3)]
        loss += model['loss'].forward(a3, yi)

        # Backward.
        da3 = model['loss'].backward(a3, yi)
        da2 = model['relu'].backward(a2, da3)
        da1 = model['layer2'].backward(a1, da2)
        da0 = model['relu'].backward(a0, da1)
        model['layer1'].backward(xi, da0)

        model['layer1'].weight = model['layer1'].weight - learningRate * model['layer1'].gradWeight
        model['layer1'].bias = model['layer1'].bias - learningRate * model['layer1'].gradBias
        
        model['layer2'].weight = model['layer2'].weight - learningRate * model['layer2'].gradWeight
        model['layer2'].bias = model['layer2'].bias - learningRate * model['layer2'].gradBias
    
    if epoch % 100 == 0: print('epoch[%d] = %.8f' % (epoch, loss / dataset_size))

Accuracy

In [ ]:
def Accuracy(yActual, yPred):
    return float(len([i for i in range(len(yPred)) if abs(yActual[i]-yPred[i])<=1])/float(len(yPred)))